In [1]:
import attr
import numpy as np
import xsimlab as xs


So it is very important, that the index is variable, and can vary between created instances of class
# I extended the code example below to show what still does not work: variable dimensions

I think the only way this can actually work, is if either the handling of write_index_vars() function within xsimlab/drivers.py is modified, or the actual index variable name is changed by the make_process, not just the dims. I will experiment some more, any help will be very much appreciated.


In [90]:
dict(A.__dict__)

{'__module__': '__main__',
 'x': _CountingAttr(counter=54, _default=NOTHING, repr=False, eq=True, order=True, hash=None, init=False, metadata={'var_type': <VarType.INDEX: 'index'>, 'dims': (('x',),), 'intent': <VarIntent.OUT: 'out'>, 'groups': (), 'attrs': {}, 'description': '', 'encoding': {}}),
 'dim': _CountingAttr(counter=55, _default=NOTHING, repr=True, eq=True, order=True, hash=None, init=True, metadata={'var_type': <VarType.VARIABLE: 'variable'>, 'dims': ((),), 'intent': <VarIntent.IN: 'in'>, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}}),
 'initialize': <function __main__.A.initialize(self)>,
 '__dict__': <attribute '__dict__' of 'A' objects>,
 '__weakref__': <attribute '__weakref__' of 'A' objects>,
 '__doc__': None}

In [91]:
ADICT = dict(A.__dict__)

In [92]:
ADICT['new_dim'] = xs.variable(default=1)

In [93]:
ADICT['new_dim']

_CountingAttr(counter=85, _default=1, repr=True, eq=True, order=True, hash=None, init=True, metadata={'var_type': <VarType.VARIABLE: 'variable'>, 'dims': ((),), 'intent': <VarIntent.IN: 'in'>, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}})

In [95]:
new_dim = xs.index(dims='n')

In [96]:
new_cls = type('test', A.__bases__, ADICT)
    
new_cls.new_dim.metadata['dims'] = 't'


In [97]:
def initialize_new(self):
        self.dim_x = np.arange(self.dim)  

In [98]:
new_cls.initialize = initialize_new

In [87]:
rxa = xs.process(new_cls)

xs.process_info(rxa)

<test  (xsimlab process)>
Variables:
    new_dim    [out] t 
    dim         [in]
Simulation stages:
    initialize


In [2]:
# do not decorate A with attr.s or xs.process!!
#@xs.process
class A:

    dim = xs.variable(intent='in')
    c_labels = xs.variable(intent='in')

    def initialize_postdimsetup(self):
        print('INIT of base class')

In [38]:
def make_process(base_cls, cls_name, dimvar):
    
    new_dim = xs.index(dims=dimvar)
    
    base_dict = dict(base_cls.__dict__)
    
    base_dict[dimvar] = new_dim
    
    def initialize_new(self):
        dim = getattr(self, 'dim')
        
        cls_label = getattr(self, '__xsimlab_name__')
        print(cls_label)
        c_labels = getattr(self, 'c_labels')
        
        index_list = [f"{label}-{cls_label}" for label in c_labels]
        setattr(self, dimvar, index_list)
        
        cls_here = getattr(self, '__class__')
        super(cls_here, self).initialize_postdimsetup()

    new_cls = type(cls_name, base_cls.__bases__, base_dict)
    
    new_cls.c_labels.metadata['dims'] = dimvar
    
    setattr(new_cls, 'initialize', initialize_new)
    
    return xs.process(new_cls)

AttributeError: type object 'MyModel' has no attribute 'foo'

In [217]:
xs.process_info(model)

<MyModel  (xsimlab process)>
Variables:
    *empty*
Simulation stages:
    *no stage implemented*


In [211]:
A = MyModel.create_flux( dimvar='X')
xs.process(A)#xs.process_info(A)

__main__.MyModel

In [165]:
mod = xs.Model({'A':a})

In [166]:
in_DS = xs.create_setup(model=mod,
                           # necessary for xsimlab
                           clocks={'clock': [0, 1]},
                           input_vars={
                               'A__c_labels':['A','B'],
                           }
                       )

KeyError: "('A', 'c_labels') is/are not valid key(s) for input variables in model <xsimlab.Model (1 processes, 0 inputs)>\nA\n"

In [110]:
B = make_process(A, 'B', 'y')

In [111]:
xs.process_info(B)

<B  (xsimlab process)>
Variables:
    dim          [in]
    c_labels     [in] y 
    y           [out] ('y',) 
Simulation stages:
    initialize


In [112]:
C = make_process(A, 'C', 'z')

In [113]:
Q = make_process(A, 'Q', 'q')

In [43]:
C

__main__.C

In [44]:
xs.process_info(C)

<C  (xsimlab process)>
Variables:
    dim          [in]
    c_labels     [in] z 
    z           [out] ('z',) 
Simulation stages:
    initialize


In [45]:

Model = xs.Model({'C':C,'B':B, 'Q':Q})

In [46]:
in_DS = xs.create_setup(model=Model,
                           # necessary for xsimlab
                           clocks={'clock': [0, 1]},
                           input_vars={
                               'C__dim':1,
                               'C__c_labels':['A','B'],
                               'B__dim':5,
                               'B__c_labels':['A'],
                               'Q__dim':5,
                               'Q__c_labels':['A'],
                           }
                       )

In [47]:
with Model:
    out_DS = in_DS.xsimlab.run()

C
INIT of base class
B
INIT of base class
Q
INIT of base class


In [48]:
out_DS

<xarray.Dataset>
Dimensions:      (clock: 2, q: 1, y: 1, z: 2)
Coordinates:
  * clock        (clock) int64 0 1
  * q            (q) <U3 'A-Q'
  * y            (y) <U3 'A-B'
  * z            (z) <U3 'A-C' 'B-C'
Data variables:
    B__c_labels  (y) <U1 'A'
    B__dim       int64 5
    C__c_labels  (z) <U1 'A' 'B'
    C__dim       int64 1
    Q__c_labels  (q) <U1 'A'
    Q__dim       int64 5